In [1]:
import pandas as pd

# initializing marker
marker = pd.read_excel(r"D:\MINAS LQR SPLIT.xlsx", 
                       sheet_name = "Marker")
marker = pd.pivot_table(marker, 
                        index = "Well", 
                        columns = "Surface", 
                        values = "MD")
marker = marker.reset_index().rename_axis(None, axis = 1)

# completion event
perfo = pd.read_excel(r"D:\MINAS LQR SPLIT.xlsx", 
                      sheet_name = "Komplesi")

# list of sands
sand = pd.read_excel(r"D:\MINAS LQR SPLIT.xlsx", 
                       sheet_name = "Sand")['Marker'].tolist()

# merging the marker and perfo data
perfo_wells = perfo.merge(marker,
                          how = 'inner', 
                          left_on = 'UWI', 
                          right_on = 'Well')

perfo_wells = perfo_wells[['UWI','DATE','Perf Status','Perf Top (ftMD)','Perf Base (ftMD)']+sand]

In [3]:
# Markering Telisa and Sihapas
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# blank dataframe to contain marker from top completion and bottom completion
df = pd.DataFrame()

# start markering!
for x,y in list(zip(['Perf Top (ftMD)', 'Perf Base (ftMD)'], ['TOP_WAY', 'BOTTOM_WAY'])):

    # initialising iterator
    i = 0

    # looping through all wells
    for well in list(perfo_wells['UWI'].unique()):
        
        if well not in list(marker['Well'].unique()):

            pass

        else:

            # initialising temporary dataframe
            temp = perfo_wells[perfo_wells['UWI'] == well]
            temp = temp.dropna(axis = 1)
            sand_list = list(temp.columns[5:])
            
            j = 0
            while j < len(temp):

                for k in range(0, len(sand_list)):
                    
                    # adressing one marker only
                    if len(sand_list) == 1:
                        if temp[x][i+j] >= temp[sand_list[k]][i+j]:
                            df.loc[i+j,y] = str(sand[sand.index(sand_list[k])])
                            break
                        else:
                            df.loc[i+j,y] = str(sand[sand.index(sand_list[k]) - 1])
                            break
                    else:
                        # addressing first sand
                        if sand_list[k] == sand_list[0]:
                            if temp[x][i+j] < temp[sand_list[k]][i+j]:
                                df.loc[i+j,y] = str(sand[sand.index(sand_list[k]) - 1])
                                break
                    
                        # addressing last sand
                        elif sand_list[k] == sand_list[-1]:

                            if temp[x][i+j] >= temp[sand_list[k]][i+j]:
                                df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k])])
                                break
                            else:
                                df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k]) - 1])
                                break
                        
                        # addressing all sand except first and last sand
                        else:
                            if temp[x][i+j] < temp[sand_list[k]][i+j]:
                                df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k]) - 1])
                                break
                
                j += 1 
                
            i += j

# dropping marker columns
perfo_wells['TOP_WAY'] = df['TOP_WAY']
perfo_wells['BOTTOM_WAY'] = df['BOTTOM_WAY']
perfo_wells['MARKER'] = np.nan # containing final marker

i = 0

# markering final marker
for well in list(perfo_wells['UWI'].unique()):
    
    if well not in list(marker['Well'].unique()):

        pass

    else:

        # initialising temporary dataframe
        temp = perfo_wells[perfo_wells['UWI'] == well]
        temp = temp.dropna(axis = 1)
        temp.drop(['TOP_WAY', 'BOTTOM_WAY'], axis=1, inplace=True)
        sand_list = list(temp.columns[5:])
        
        j = 0
        while j < len(temp):
            
            if (
                (perfo_wells['TOP_WAY'][i+j] not in sand_list and  perfo_wells['BOTTOM_WAY'][i+j] not in sand_list) or 
                (perfo_wells['TOP_WAY'][i+j] not in sand_list or perfo_wells['BOTTOM_WAY'][i+j] not in sand_list)
                ):
                perfo_wells['MARKER'][i+j] = sand[sand.index(perfo_wells['TOP_WAY'][i+j]):sand.index(perfo_wells['BOTTOM_WAY'][i+j])+1]
            else:
                perfo_wells['MARKER'][i+j] = sand_list[sand_list.index(perfo_wells['TOP_WAY'][i+j]):sand_list.index(perfo_wells['BOTTOM_WAY'][i+j])+1]

            j += 1
        
        i += j

In [4]:
perfo_wells

,UWI,DATE,Perf Status,Perf Top (ftMD),Perf Base (ftMD),TETX,TET1,TET2,TET3,TET4,...,B2U1,BND,DU4,DU3,DU2,DU1,MNS,TOP_WAY,BOTTOM_WAY,MARKER
0,MINA00001,1943-12-10,perforation,2110.0,2116.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BKX,BKX,[BKX]
1,MINA00001,1943-12-10,perforation,2126.0,2146.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BKA1,A1U2,"[BKA1, A1U2]"
2,MINA00001,1943-12-10,perforation,2211.0,2218.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
3,MINA00001,1943-12-10,perforation,2244.0,2251.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
4,MINA00001,1943-12-10,perforation,2277.0,2283.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201350,MINA02111D1,2023-12-28,perforation,2618.0,2624.0,NaN,NaN,NaN,NaN,NaN,...,2797.446181,2830.546181,2832.546181,2840.046181,2844.746181,2860.046181,2896.746181,A1L1,A1L1,[A1L1]
201351,MINA02111D1,2023-12-28,perforation,2702.0,2705.0,NaN,NaN,NaN,NaN,NaN,...,2797.446181,2830.546181,2832.546181,2840.046181,2844.746181,2860.046181,2896.746181,B1U2,B1U2,[B1U2]
201352,MINA02112D1,2023-12-22,perforation,2230.0,2234.0,NaN,NaN,NaN,NaN,NaN,...,2275.018830,2288.118830,2303.918830,2311.718830,2336.918830,2359.718830,2434.218830,B2U3,B2U3,[B2U3]
201353,MINA02112D1,2023-12-22,perforation,2302.0,2304.0,NaN,NaN,NaN,NaN,NaN,...,2275.018830,2288.118830,2303.918830,2311.718830,2336.918830,2359.718830,2434.218830,BND,DU4,"[BND, DU4]"


In [67]:
perfo_wells.to_excel(r"D:\perfo_wells.xlsx")

In [5]:
# function to process the perforation and squeeze events
def squeeze_machine(tp,bp,pf,pd,ts,bs,sf,sd,marker_temp):
    
    a = 0
    # looping through every squeezed event   
    while a < len(sd):

        #checking each squeeze event to each perforation event
        b = 0
        while b < len(pd):
            
            # handling partial squeeze
            if (sd[a] >= pd[b]):
                
                # changing top perforation and deleting squeezed sand
                if ((ts[a] == tp[b]) and  bs[a] < bp[b]):
                    tp[b] = bs[a]
                    if (len(pf[b])==len(sf[a])):
                        pf[b] = [pf[b][-1]]
                    elif (len(pf[b]) > len(sf[a])):
                        if tp[b] < marker_temp.loc[0, pf[b][len(sf[a])]]:
                            pf[b] = [pf[b][len(sf[a])-1]] + [x for x in pf[b] if x not in sf[a]]
                        else:
                            pf[b] = [x for x in pf[b] if x not in sf[a]]

                # changing bottom perforation and deleting squeezed sand
                elif ((ts[a] > tp[b]) and  bs[a] == bp[b]):
                    bp[b] = ts[a]
                    if (len(pf[b])==len(sf[a])):
                        pf[b] = [pf[b][0]] 
                    elif (len(pf[b]) > len(sf[a])):
                        if bp[b] > marker_temp.loc[0, pf[b][len(sf[a])]]:
                            pf[b] = [pf[b][len(sf[a])]] + [x for x in pf[b] if x not in sf[a]]
                        else:
                            pf[b] = [x for x in pf[b] if x not in sf[a]]
       
            # deleting perforation event (date, top perforation, bottom perforation and sand) that squeezed
            if ((ts[a] <= tp[b] and bs[a] >= bp[b]) 
                and sd[a] >= pd[b]):
                del pf[b]
                del tp[b]
                del bp[b]
                del pd[b] 
            else:
                b += 1

        a += 1
    
    return tp,bp,pf

# function to count perforated sand at certain event of production
def count_sand(res, pf):

    if len(pf) > 0:
        
        x = 0
        for i in range(0, len(pf)):
            if res in pf[i]:
                x += 1        
        return x

    else:
        return 0
    
def writing_perforation (production, sands, i):

    x = 0
    for key, value in sands.items():
        if value > 0:
            production.loc[i, key]= "p"
        x += 1

In [6]:
perfo_wells = perfo_wells.sort_values(by = ['UWI','DATE'])

In [7]:
perfo_wells

,UWI,DATE,Perf Status,Perf Top (ftMD),Perf Base (ftMD),TETX,TET1,TET2,TET3,TET4,...,B2U1,BND,DU4,DU3,DU2,DU1,MNS,TOP_WAY,BOTTOM_WAY,MARKER
0,MINA00001,1943-12-10,perforation,2110.0,2116.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BKX,BKX,[BKX]
1,MINA00001,1943-12-10,perforation,2126.0,2146.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BKA1,A1U2,"[BKA1, A1U2]"
2,MINA00001,1943-12-10,perforation,2211.0,2218.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
3,MINA00001,1943-12-10,perforation,2244.0,2251.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
4,MINA00001,1943-12-10,perforation,2277.0,2283.0,1683.76,1724.76,1760.76,1807.76,1852.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1U2,A1U2,[A1U2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201350,MINA02111D1,2023-12-28,perforation,2618.0,2624.0,NaN,NaN,NaN,NaN,NaN,...,2797.446181,2830.546181,2832.546181,2840.046181,2844.746181,2860.046181,2896.746181,A1L1,A1L1,[A1L1]
201351,MINA02111D1,2023-12-28,perforation,2702.0,2705.0,NaN,NaN,NaN,NaN,NaN,...,2797.446181,2830.546181,2832.546181,2840.046181,2844.746181,2860.046181,2896.746181,B1U2,B1U2,[B1U2]
201352,MINA02112D1,2023-12-22,perforation,2230.0,2234.0,NaN,NaN,NaN,NaN,NaN,...,2275.018830,2288.118830,2303.918830,2311.718830,2336.918830,2359.718830,2434.218830,B2U3,B2U3,[B2U3]
201353,MINA02112D1,2023-12-22,perforation,2302.0,2304.0,NaN,NaN,NaN,NaN,NaN,...,2275.018830,2288.118830,2303.918830,2311.718830,2336.918830,2359.718830,2434.218830,BND,DU4,"[BND, DU4]"


In [10]:
# load data
marker = perfo_wells
prod = pd.read_excel(r"D:\Minas LQR SPLIT.xlsx", sheet_name = "Produksi")

In [11]:
prod

,DAY,MONTH,YEAR,DATE,WINJ,OIL,WATER,UPTIME,WELL,UWI,...,B2U4,B2U3,B2U2,B2U1,DU5,DU4,DU3,DU2,DU1,MNS
0,1.0,1.0,1944.0,1944-01-01,0.0,323.195,0.0,1.0,6D55,MINA00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2.0,1944.0,1944-02-01,0.0,344.382,0.0,1.0,6D55,MINA00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,3.0,1944.0,1944-03-01,0.0,321.137,0.0,1.0,6D55,MINA00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,4.0,1944.0,1944-04-01,0.0,330.783,0.0,1.0,6D55,MINA00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,5.0,1944.0,1944-05-01,0.0,319.092,0.0,1.0,6D55,MINA00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604014,NaN,NaN,NaN,2023-12-31,NaN,NaN,NaN,NaN,NaN,MINA02108D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604015,NaN,NaN,NaN,2023-12-31,NaN,NaN,NaN,NaN,NaN,MINA02109D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604016,NaN,NaN,NaN,2023-12-31,NaN,NaN,NaN,NaN,NaN,MINA02110D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604017,NaN,NaN,NaN,2023-12-31,NaN,NaN,NaN,NaN,NaN,MINA02111D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# create blank dataframe to store the result
final = pd.DataFrame()

# starting by looping through all wells from production data
for uwi in list(prod['UWI'].unique()):

    # checking wether the selected well is in the marker data or not
    if uwi in list(marker['UWI'].unique()):
        
        # partiliazing marker data 
        tempmarker = marker[marker['UWI']==uwi]
        tempmarker = tempmarker.dropna(axis = 1)
        tempmarker = tempmarker.reset_index(drop=True)

        # partializing production data
        temp_prod = prod[prod['UWI']==uwi]
        temp_prod = temp_prod.reset_index(drop=True)

        # looping through selected well from production data
        index = 0
        
        while index < len(temp_prod):

            # initialize starting variable
            x = 0

            # initialize empty array (perforation sand, squeeze sand, top perforation, bottom perforation, _
            # _ top squeeze, bottom squeeze, perforation date, squeeze date)
            PF,SF,TP,BP,TS,BS,PD,SD = [],[],[],[],[],[],[],[] 
            
            # checking the production data through marker data
            while x < len(tempmarker):

                # dates in every single timestep in production data is compared to all of the dates data that meet the condition
                if (
                    (temp_prod['DATE'][index].year > tempmarker['DATE'][x].year) or 
                    ((temp_prod['DATE'][index].year == tempmarker['DATE'][x].year) and 
                    (temp_prod['DATE'][index].month >= tempmarker['DATE'][x].month))
                    ):

                    # perforation event 
                    if tempmarker['Perf Status'][x] == "perforation":
                        
                        # appending perforation event data
                        PF.append(tempmarker['MARKER'][x])
                        TP.append(tempmarker['Perf Top (ftMD)'][x])
                        BP.append(tempmarker['Perf Base (ftMD)'][x])
                        PD.append(tempmarker['DATE'][x])

                    # squeeze event
                    elif tempmarker['Perf Status'][x] == "squeeze":

                        # appending squeeze event data
                        SF.append(tempmarker['MARKER'][x])
                        TS.append(tempmarker['Perf Top (ftMD)'][x])
                        BS.append(tempmarker['Perf Base (ftMD)'][x])
                        SD.append(tempmarker['DATE'][x])
                    
                    else:
                        pass
        
                    x += 1

                else:
                    break

            # squeeze handling 
            squeeze_machine(TP,BP,PF,PD,TS,BS,SF,SD,tempmarker)

            # creating dictionary that hold number of perforated sands
            sands = {}
            for minas in sand:
                sands[minas] = count_sand(minas, PF)

            # write "p" on the dataframe to mark perforated sand
            writing_perforation(temp_prod, sands, index)
            
            index += 1

    else:
        
        continue

     # print the result in the terminal
    print(f'Well : {uwi} | index : {index}')
    
    # concating the result of each well after the "p" writing process
    final = pd.concat([final,temp_prod])

Well : MINA00001 | index : 603
Well : MINA00002 | index : 881
Well : MINA00003 | index : 879
Well : MINA00004 | index : 585
Well : MINA00005 | index : 882
Well : MINA00006 | index : 585
Well : MINA00007 | index : 873
Well : MINA00008 | index : 805
Well : MINA00009 | index : 872
Well : MINA00010 | index : 871
Well : MINA00011 | index : 870
Well : MINA00012 | index : 869
Well : MINA00013 | index : 869
Well : MINA00014 | index : 867
Well : MINA00015 | index : 866
Well : MINA00016 | index : 866
Well : MINA00017 | index : 862
Well : MINA00018 | index : 862
Well : MINA00019 | index : 804
Well : MINA00020 | index : 861
Well : MINA00021 | index : 861
Well : MINA00022 | index : 861
Well : MINA00023 | index : 845
Well : MINA00024 | index : 860
Well : MINA00025 | index : 778
Well : MINA00026 | index : 814
Well : MINA00027 | index : 859
Well : MINA00028 | index : 859
Well : MINA00029 | index : 553
Well : MINA00030 | index : 658
Well : MINA00031 | index : 550
Well : MINA00032 | index : 760
Well : M

In [13]:
final.to_excel(r'D:\Auto-P_MINAS_new_version_update_final.xlsx')